### Collecting News,article on Climate Language

In [ ]:
from gnews import GNews
import pandas as pd

def initialize_news_client(language='en', country='NP', max_results=50):
    """Initialize GNews client with language, country, and max results."""
    news = GNews(language=language, country=country, max_results=max_results)
    return news

def search_news_articles(news_client, keyword):
    """Search news articles related to a keyword."""
    articles = news_client.get_news(keyword)
    return articles

def extract_article_info(articles):
    """Extract useful fields from raw article data."""
    extracted_data = []
    for article in articles:
        extracted_data.append({
            'title': article.get('title', ''),
            'description': article.get('description', ''),
            'published_date': article.get('published date', ''),
            'url': article.get('url', ''),
            'publisher': article.get('publisher', {}).get('title', '')
        })
    return extracted_data

def save_articles_to_csv(extracted_data, filename="climate_news_articles.csv"):
    """Save extracted articles into a CSV file."""
    df = pd.DataFrame(extracted_data)
    df.to_csv(filename, index=False)
    print(f"Saved {len(df)} articles to {filename}")

# 🛠 Example Usage:
if __name__ == "__main__":pip install nltk

    news_client = initialize_news_client(language='en', country='NP', max_results=50)
    articles = search_news_articles(news_client, keyword="climate change")
    extracted_data = extract_article_info(articles)
    save_articles_to_csv(extracted_data)


Saved 50 articles to climate_news_articles.csv


### Sentiment Analysis

In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

def initialize_sentiment_analyzer():
    """Initialize VADER Sentiment Analyzer."""
    analyzer = SentimentIntensityAnalyzer()
    return analyzer

def analyze_sentiment(text, analyzer):
    """Analyze sentiment polarity scores of a given text."""
    if not isinstance(text, str):
        return None
    score = analyzer.polarity_scores(text)
    return score

def assign_sentiment_label(score):
    """Assign Positive, Negative, Neutral labels based on compound score."""
    if score is None:
        return "Unknown"
    compound = score['compound']
    if compound >= 0.05:
        return "Positive"
    elif compound <= -0.05:
        return "Negative"
    else:
        return "Neutral"

def apply_sentiment_analysis(input_csv="../news/climate_news_articles.csv", output_csv="../news/climate_news_with_sentiment.csv"):
    """Apply sentiment analysis on the news articles CSV and save results."""
    df = pd.read_csv(input_csv)
    analyzer = initialize_sentiment_analyzer()
    
    # Analyze sentiment
    df['sentiment_score'] = df['description'].apply(lambda x: analyze_sentiment(x, analyzer))
    df['sentiment_label'] = df['sentiment_score'].apply(assign_sentiment_label)
    
    # Save the new dataframe
    df.to_csv(output_csv, index=False)
    print(f"Sentiment analysis completed and saved to {output_csv}")

# 🛠 Example Usage:
if __name__ == "__main__":
    apply_sentiment_analysis()


Sentiment analysis completed and saved to ../news/climate_news_with_sentiment.csv


### NER and Extracting location and name 

In [5]:

!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     --- ------------------------------------ 1.0/12.8 MB 1.7 MB/s eta 0:00:07
     ----- ---------------------------------- 1.8/12.8 MB 2.5 MB/s eta 0:00:05
     --------- ------------------------------ 3.1/12.8 MB 3.3 MB/s eta 0:00:03
     ------------- -------------------------- 4.2/12.8 MB 3.8 MB/s eta 0:00:03
     ----------------- ---------------------- 5.5/12.8 MB 4.0 MB/s eta 0:00:02
     -------------------- ------------------- 6.6/12.8 MB 4.2 MB/s eta 0:00:02
     ----------------------- ---------------- 7.6/12.8 MB 4.3 MB/s eta 0:00:02
     --------------------------- ------------ 8.7/12.8 MB 4.3 MB/s eta 0:00:01
     ------------------------------ --------- 9.7/12.8 MB 4.4 MB/s eta 0:00:01
 

In [11]:
import spacy
import pandas as pd

def initialize_spacy_model(model_name='en_core_web_sm'):
    """Initialize the spaCy NLP model."""
    nlp = spacy.load(model_name)
    return nlp

def extract_named_entities(text, nlp_model):
    """Extract named entities like location, organization, event from the text."""
    if not isinstance(text, str):
        return []
    doc = nlp_model(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

def filter_entities_by_type(entities, types=["GPE", "LOC", "ORG", "EVENT"]):
    """Filter entities based on specified types (like Location, Organization, Event)."""
    return [ent for ent in entities if ent[1] in types]

def apply_ner(input_csv="../news/climate_news_with_sentiment.csv", output_csv="../news/climate_news_with_ner.csv"):
    """Apply NER on news articles and save the enriched file."""
    df = pd.read_csv(input_csv)
    nlp_model = initialize_spacy_model()

    df['named_entities'] = df['description'].apply(lambda x: filter_entities_by_type(extract_named_entities(x, nlp_model)))

    df.to_csv(output_csv, index=False)
    print(f"Named Entity Recognition completed and saved to {output_csv}")

# 🛠 Example Usage:
if __name__ == "__main__":
    apply_ner()
    


Named Entity Recognition completed and saved to ../news/climate_news_with_ner.csv


In [6]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')



[nltk_data] Downloading package punkt to C:\Users\SHYAM
[nltk_data]     PANDIT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\SHYAM
[nltk_data]     PANDIT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Topic Modelig system

In [13]:

import nltk
import pandas as pd
import gensim
from gensim import corpora

nltk.data.path.append(r"C:\Users\SHYAM PANDIT\AppData\Roaming\nltk_data")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re


def preprocess_text(text):
    """Clean and tokenize text for topic modeling without nltk punkt."""
    import nltk
    from nltk.corpus import stopwords
    import re

    stop_words = set(stopwords.words('english'))

    if not isinstance(text, str):
        text = str(text)

    tokens = re.findall(r'\b\w+\b', text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens




def create_dictionary_and_corpus(texts):
    """Create dictionary and corpus for LDA model."""
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    return dictionary, corpus

def build_lda_model(corpus, dictionary, num_topics=5):
    """Build and return the LDA model."""
    lda_model = gensim.models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42, passes=15)
    return lda_model

def print_topics(lda_model, num_words=5):
    """Print the topics discovered by the LDA model."""
    topics = lda_model.print_topics(num_words=num_words)
    for idx, topic in topics:
        print(f"Topic {idx}: {topic}")

def apply_topic_modeling(input_csv="../news/climate_news_with_ner.csv", text_column="description", num_topics=5):
    """Apply topic modeling on news articles."""
    df = pd.read_csv(input_csv)
    
    # Check for NaN or empty values in the text column
    if df[text_column].isnull().any():
        print(f"Warning: Some rows in '{text_column}' column have NaN values.")
        df = df.dropna(subset=[text_column])  # Remove rows with NaN values
    
    texts = df[text_column].apply(preprocess_text)
    
    dictionary, corpus = create_dictionary_and_corpus(texts)
    lda_model = build_lda_model(corpus, dictionary, num_topics=num_topics)
    
    print("Generated Topics:")
    print_topics(lda_model)
    
    return lda_model, dictionary, corpus


if __name__ == "__main__":
    # Apply topic modeling on the enriched CSV
  lda_model, dictionary, corpus = apply_topic_modeling()



Generated Topics:
Topic 0: 0.055*"climate" + 0.042*"change" + 0.015*"new" + 0.015*"pope" + 0.015*"francis"
Topic 1: 0.064*"climate" + 0.051*"change" + 0.014*"un" + 0.014*"action" + 0.014*"leaders"
Topic 2: 0.083*"climate" + 0.062*"change" + 0.032*"news" + 0.011*"us" + 0.011*"making"
Topic 3: 0.066*"climate" + 0.052*"change" + 0.037*"arizona" + 0.030*"republic" + 0.016*"environment"
Topic 4: 0.060*"climate" + 0.047*"change" + 0.014*"global" + 0.014*"new" + 0.014*"francis"


### Text Summarization

In [ ]:
import pandas as pd
from transformers import pipeline

def initialize_summarizer(model_name="facebook/bart-large-cnn"):
    """Initialize Huggingface summarization pipeline."""
    summarizer = pipeline("summarization", model=model_name)
    return summarizer

def summarize_text(text, summarizer, min_length=30, max_length=120):
    """Summarize a given text."""
    if not isinstance(text, str) or len(text.split()) < 30:
        return text  # Skip very short texts
    
    summary = summarizer(text, min_length=min_length, max_length=max_length, do_sample=False)
    return summary[0]['summary_text']

def apply_text_summarization(input_csv="../news/climate_news_with_ner.csv", output_csv="../news/climate_news_with_summary.csv", text_column="description"):
    """Apply summarization on news articles and save."""
    df = pd.read_csv(input_csv)
    summarizer = initialize_summarizer()

    df['summary'] = df[text_column].apply(lambda x: summarize_text(x, summarizer))

    df.to_csv(output_csv, index=False)
    print(f"Summarization completed and saved to {output_csv}")

# 🛠 Example Usage:
if __name__ == "__main__":
    apply_text_summarization()


c:\Users\SHYAM PANDIT\Omdena Assignment\assignment 7\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\SHYAM PANDIT\Omdena Assignment\assignment 7\.venv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SHYAM PANDIT\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to ru

Summarization completed and saved to ../news/climate_news_with_summary.csv


### Multilingual Nepali Language

In [20]:
!pip install gnews

  Using cached gnews-0.4.1-py3-none-any.whl.metadata (19 kB)
  Using cached feedparser-6.0.11-py3-none-any.whl.metadata (2.4 kB)
  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached sgmllib3k-1.0.0.tar.gz (5.8 kB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached gnews-0.4.1-py3-none-any.whl (18 kB)
Using cached feedparser-6.0.11-py3-none-any.whl (81 kB)
Using cached dnspython-2.7.0-py3-none-any.whl (313 kB)
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6103 sha256=f042589a0b85700940df47575bbaf401de4559f957b432eb8e8091c248c4183e
  Stored in directory: c:\users\shyam pandit\appdata\local\pip

In [21]:
from gnews import GNews
import pandas as pd

def initialize_news_client_nepali(language='ne', country='NP', max_results=50):
    """Initialize GNews client for Nepali language."""
    news = GNews(language=language, country=country, max_results=max_results)
    return news

def search_nepali_climate_news(news_client, keyword="जलवायु परिवर्तन"):
    """Search Nepali climate-related news articles."""
    articles = news_client.get_news(keyword)
    return articles

def extract_article_info_nepali(articles):
    """Extract useful fields from Nepali news articles."""
    extracted_data = []
    for article in articles:
        extracted_data.append({
            'title': article.get('title', ''),
            'description': article.get('description', ''),
            'published_date': article.get('published date', ''),
            'url': article.get('url', ''),
            'publisher': article.get('publisher', {}).get('title', '')
        })
    return extracted_data

def save_nepali_articles_to_csv(extracted_data, filename="../news/nepali_climate_news_articles.csv"):
    """Save extracted Nepali articles into a CSV file."""
    df = pd.DataFrame(extracted_data)
    df.to_csv(filename, index=False, encoding='utf-8-sig')  # Save properly in Nepali
    print(f"Saved {len(df)} Nepali articles to {filename}")

# 🛠 Example Usage:
if __name__ == "__main__":
    news_client = initialize_news_client_nepali(language='ne', country='NP', max_results=50)
    articles = search_nepali_climate_news(news_client, keyword="जलवायु परिवर्तन")  # "Climate Change" in Nepali
    extracted_data = extract_article_info_nepali(articles)
    save_nepali_articles_to_csv(extracted_data)


Saved 50 Nepali articles to nepali_climate_news_articles.csv


In [25]:
from transformers import MarianMTModel, MarianTokenizer, pipeline

def load_translation_model(src_lang="en", tgt_lang="np"):
    """Load translation model for Nepali to English."""
    model_name = f'Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return tokenizer, model

def translate_nepali_to_english(text, tokenizer, model):
    """Translate Nepali text to English."""
    if not isinstance(text, str) or len(text.strip()) == 0:
        return ""
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

def initialize_multilingual_summarizer(model_name="facebook/bart-large-cnn"):
    """Initialize a summarizer model (already in English)."""
    summarizer = pipeline("summarization", model=model_name)
    return summarizer

def summarize_nepali_text(text, nepali_tokenizer, nepali_model, summarizer, min_length=30, max_length=120):
    """Translate Nepali → English → Summarize."""
    english_text = translate_nepali_to_english(text, nepali_tokenizer, nepali_model)
    if len(english_text.split()) < 30:
        return english_text  

    summary = summarizer(english_text, min_length=min_length, max_length=max_length, do_sample=False)
    return summary[0]['summary_text']

def apply_multilingual_processing(input_csv="../news/nepali_climate_news_articles.csv", output_csv="../news/nepali_climate_news_summary.csv", text_column="description"):
    """Full pipeline: Translate Nepali text → Summarize → Save."""
    df = pd.read_csv(input_csv)

    nepali_tokenizer, nepali_model = load_translation_model()
    summarizer = initialize_multilingual_summarizer()

    df['english_translation'] = df[text_column].apply(lambda x: translate_nepali_to_english(x, nepali_tokenizer, nepali_model))
    df['summary'] = df['english_translation'].apply(lambda x: summarizer(x, min_length=30, max_length=120, do_sample=False)[0]['summary_text'] if len(x.split()) > 30 else x)

    df.to_csv(output_csv, index=False)
    print(f"Multilingual processing completed and saved to {output_csv}")

# 🛠 Example Usage
if __name__ == "__main__":
    apply_multilingual_processing()


ImportError: 
MarianTokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.
